# Projeto 1 - módulo 6

## Precificação dinâmica - e-commerce

### Mercari Price Suggestion Challenge - Kaggle

Mercari é um site de revenda de produtos online. Uma dos desafios desse tipo de plataforma é auxiliar o usuário, muitas vezes com pouco conhecimento de vendas, a determinar um preço para os seus produtos de modo a maximizar as chances de venda.

### Sobre este projeto

O presente projeto tem o objetivo de desenvolver um algoritmo que identifique produtos já vendidos similares e sugira ao usuário um preço ótimo para novos produtos cadastrados.


### Preparação do ambiente

Para este projeto, acesse o link https://www.kaggle.com/competitions/mercari-price-suggestion-challenge/overview 


In [1]:
import numpy as np
import pandas as pd

In [2]:
base = pd.read_csv ('/content/drive/MyDrive/Blue Edtech/Módulo 6/Projeto 1 - Os outlieres/train_data_prep01.csv') 

In [3]:
base

,name,category_1,category_2,category_3,item_condition_id,brand_name,price,shipping,item_description,date,stock
0,mlb cincinnati reds t shirt size xl,Men,Tops,T-shirts,3,NaN,10.0,1,No description yet,18-6-2018,27
1,razer blackwidow chroma keyboard,Electronics,Computers & Tablets,Components & Parts,3,Razer,52.0,0,This keyboard is in great condition and works ...,18-3-2018,15
2,ava viv blouse,Women,Tops & Blouses,Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,26-10-2018,14
3,leather horse statues,Home,Home Décor,Home Décor Accents,1,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,21-2-2018,1
4,24k gold plated rose,Women,Jewelry,Necklaces,1,NaN,44.0,0,Complete with certificate of authenticity,17-4-2018,13
...,...,...,...,...,...,...,...,...,...,...,...
1481656,free people inspired dress,Women,Dresses,Mid-Calf,2,Free People,20.0,1,"Lace, says size small but fits medium perfectl...",6-5-2018,20
1481657,little mermaid handmade dress,Kids,Girls 2T-5T,Dresses,2,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,12-12-2018,4
1481658,day fix containers and eating plan,Sports & Outdoors,Exercise,Fitness accessories,2,NaN,12.0,0,"Used once or twice, still in great shape.",7-10-2018,4
1481659,world markets lanterns,Home,Home Décor,Home Décor Accents,3,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...,21-5-2018,3


In [4]:
base['texto'] = base['name'] + ' ' + base['category_1'] + ' ' + base['category_2'] + ' ' + base['category_3'] + ' ' + base['item_condition_id'].astype(str) + base['brand_name'] + ' ' + base['shipping'] + ' ' + base['item_description']

In [5]:
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# extraindo características

textos = base[['texto']]

stop_words = stopwords.words('english')
tf = TfidfVectorizer(analyzer = 'word', min_df=0, stop_words = stop_words)
tfidv_matrix = tf.fit_transform(textos['texto'].values.astype('U'))
tfidv_matrix.shape

(1481661, 134792)

In [8]:
def ecommerce(produto):

  # calculando a similaridade de coseno do vinho selecionado com os demais

  medida = cosine_similarity(tfidv_matrix[produto], tfidv_matrix)
  select = (base.index == produto)
  produto_cadastrado = base[select]

# criando a função que identifica o index dos vinhos com maiores coeficiente de similaridade e impõe umm limite de 0.5 

  limite = 0.5
  result = []

  for cnt in np.arange(10):
    ind = cnt - 10 # ind vai de -10 a -1
    index = medida[0].argsort()[ind] # retornando os índices q ordenam o array
    measure = medida[0][index]

    if index == produto:
      continue
    if (measure > limite):
      result.append([index,measure])

# criando listas com os indices e scores dos 9 produtos com as maiores similaridades em relação ao novo produto cadastrado

  indices = []
  scores = []

# resultado_parcial = recomendacoes(medida, produto) 
  for ind, scr in result:
   if ind not in indices:
     indices.append(ind)
     scores.append(scr)

# criando um datraframe com os produto identificados

   produtos_similares = pd.DataFrame()

  for ind in indices:
    select = (base.index == ind)
    df_temp = base[select]
    produtos_similares = pd.concat([produtos_similares, df_temp], axis = 0)

  produtos_similares['score'] = scores

  return produtos_similares

In [55]:
# Teste você mesmo!

produto = int(input('Digite índice do produto: \n'))
select = (base.index == produto)
produto_cadastrado = base[select]
produto_cadastrado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

Digite índice do produto: 
100000


,name,category_1,category_2,category_3,item_description,item_condition_id,brand_name,price
100000,h m hoodie,Men,Sweats & Hoodies,Hoodie,Worn only once.,2,H&M,25.0


In [56]:
# lista de produtos similares

resultado = ecommerce(produto)
resultado[['name', 'category_1', 'category_2', 'category_3', 'item_description', 'item_condition_id', 'brand_name', 'price']]

,name,category_1,category_2,category_3,item_description,item_condition_id,brand_name,price
885941,hoodie,Men,Sweats & Hoodies,Hoodie,No description yet,3,Nike,21.0
86133,nike hoodie,Men,Sweats & Hoodies,Hoodie,Men's Medium,3,Nike,12.0
290096,adidas hoodie,Men,Sweats & Hoodies,Hoodie,Medium men's hoodie,2,Adidas,17.0
112218,nike hoodie,Men,Sweats & Hoodies,Hoodie,This is nike hoodie is in great condition and ...,2,Nike,20.0
861815,nike hoodie,Men,Sweats & Hoodies,Hoodie,Brand new with tags!! Very warm.,1,Nike,25.0
805303,men s nike hoodie,Men,Sweats & Hoodies,Hoodie,Great condition / no flaws Size large,3,Nike,25.0
483833,hoodie,Men,Sweats & Hoodies,Hoodie,Never worn.,2,Adidas,20.0
713993,men s h m grey hoodie medium,Men,Sweats & Hoodies,Hoodie,Great shape! Only worn a few times. Size medium.,2,H&M,12.0
1446462,nike hoodie,Men,Sweats & Hoodies,Hoodie,Very comfortable,1,Nike,25.0


In [57]:
preco_min = resultado['price'].min()
preco_max = resultado['price'].max()
preco_medio = resultado['price'].mean()
preco_moda = resultado['price'].mode()
print(preco_min, preco_max, preco_medio, preco_moda)

12.0 25.0 19.666666666666668 0    25.0
dtype: float64
